In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [133]:
general = pd.read_csv('general-info.csv')

In [137]:
# Need a function for converting dates to correct format
def convert_date(inp):
    string = str(inp)
    print(string)
    if string == 'nan':
        return np.nan
    elif re.match(r'\d{4}-\d{2}-\d{2}',string):
        return string
    else:
        return datetime.strftime(datetime.strptime(string, "%B %d, %Y"), "%Y-%m-%d")

In [138]:
general['ReleaseDate'] = general['ReleaseDate'].apply(convert_date)

2010-06-11
2013-10-04
2001-06-29
2007-11-30
2011-08-12
2010-08-27
2014-01-24
2002-10-18
2007-02-23
2002-05-03
2013-03-08
2001-11-30
2013-02-08
2015-06-19
2014-10-31
2011-09-23
2006-11-24
nan
2001-08-17
2006-04-14
2004-02-20
2002-05-17
2001-05-11
2014-08-08
2012-09-21
2015-04-08
2011-11-11
2014-02-14
1986-07-04
nan
2002-12-13
2013-11-01
2016-04-15
2015-01-30
1988-04-08
1994-03-25
2012-06-22
2013-07-05
nan
1986-04-18
1997-02-14
2006-10-27
nan
2016-07-22
2009-02-06
2014-08-15
1989-08-11
1993-02-26
2006-08-18
2014-05-23
nan
1988-12-23
1993-12-23
2016-10-14
1988-10-14
nan
1994-02-04
1995-11-10
1992-06-12
1995-10-20
2007-09-14
2009-07-31
2013-07-19
2012-02-24
1988-02-12
2014-12-05
2014-11-07
2003-11-21
2009-07-29
2006-03-31
2007-03-16
nan
2002-12-06
1991-11-22
1993-11-19
2016-04-15
2014-10-10
2012-11-30
1997-05-23
1995-10-06
2011-12-14
1992-05-29
2011-03-04
2014-08-08
2013-03-22
2009-05-08
2013-09-06
2004-04-02
2002-10-18
2007-09-21
2015-04-24
2014-02-14
2014-09-26
2009-04-03
2014-01-10
1987

In [118]:
general = general.sort_values(by=['movieID','ReleaseDate'])

In [140]:
films = list(set(general.movieID))

In [141]:
rel = {}
for f in films:
    s = general[ (general['movieID'] == f) & (general['ReleaseDate'].notnull()) ]
    if len(s) > 0:
        rel[f] = s.iloc[0]['ReleaseDate']
    else:
        rel[f] = np.nan

In [142]:
general['ReleaseDate'] = general['movieID'].map(rel)

In [145]:
general = general.drop_duplicates()

In [147]:
movies = pd.read_csv('movie_list.csv')

In [181]:
combined = movies.merge(general, indicator=True)

In [182]:
combined['AdjustedDomesticGross'] = combined['AdjustedDomesticGross'].str.replace('$','')
combined['AdjustedDomesticGross'] = combined['AdjustedDomesticGross'].str.replace(',','')
combined['AdjustedDomesticGross'] = pd.to_numeric(combined['AdjustedDomesticGross'], errors='coerce')

In [183]:
combined['ReleaseDate'] = pd.to_datetime(combined['ReleaseDate'])

In [184]:
combined['Release_Year'] = pd.DatetimeIndex(combined['ReleaseDate']).year
combined['Release_Month'] = pd.DatetimeIndex(combined['ReleaseDate']).month

In [185]:
def to_integer(s):
    if str(s) == "nan":
        return np.nan
    else:
        return int(s)

In [186]:
combined['Release_Year'] = combined['Release_Year'].apply(to_integer)
combined['Release_Month'] = combined['Release_Month'].apply(to_integer)

In [187]:
combined.sort_values(by='movieID').to_csv('films_cleaned.csv',index=False)

In [21]:
detailed = pd.read_csv('weekend-info.csv').tail(2)

In [22]:
detailed = detailed.drop_duplicates()

In [24]:
import re
f = lambda x: True if re.search('<i>',x) else False
detailed['4day'] = detailed['Week_num'].apply(f)

In [25]:
columns = list(detailed.columns)
columns.remove('Year')
columns.remove('4day')

In [26]:
for c in columns:
    detailed[c] = detailed[c].str.replace("<i>","")
    detailed[c] = detailed[c].str.replace("</i>","")

detailed['Rank'] = detailed['Rank'].str.replace("-","")
detailed['Rank'] = detailed['Rank'].str.replace(" ","")
detailed['Theaters'] = detailed['Theaters'].str.replace("-","")
detailed['Theaters'] = detailed['Theaters'].str.replace(" ","")

In [27]:
detailed['Week_num'] = detailed['Week_num'].astype('int')

In [28]:
detailed['Rank'] = pd.to_numeric(detailed['Rank'], errors = 'coerce')
detailed['Theaters'] = pd.to_numeric(detailed['Theaters'], errors = 'coerce')

In [29]:
detailed['pcChange'] = detailed['pcChange'].str.replace(r'(-)$','')
detailed['pcChange'] = detailed['pcChange'].str.replace('%','')
detailed['pcChange'] = pd.to_numeric(detailed['pcChange'], errors = 'coerce')

In [30]:
detailed['Change'] = detailed['Change'].str.replace(r'(-)$','')
detailed['Change'] = pd.to_numeric(detailed['Change'], errors = 'coerce')

In [31]:
detailed['WeekendGross'] = detailed['WeekendGross'].str.replace('$','')
detailed['WeekendGross'] = detailed['WeekendGross'].str.replace(',','')
detailed['WeekendGross'] = pd.to_numeric(detailed['WeekendGross'], errors = 'coerce')

detailed['Avg'] = detailed['Avg'].str.replace('$','')
detailed['Avg'] = detailed['Avg'].str.replace(',','')
detailed['Avg'] = pd.to_numeric(detailed['Avg'], errors = 'coerce')

detailed['Gross-to-Date'] = detailed['Gross-to-Date'].str.replace('$','')
detailed['Gross-to-Date'] = detailed['Gross-to-Date'].str.replace(',','')
detailed['Gross-to-Date'] = pd.to_numeric(detailed['Gross-to-Date'], errors = 'coerce')

In [32]:
detailed['Month_name'] = detailed['Date'].str.extract(r'^(\D{3})')

/Users/simonbedford/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [33]:
months = {'Jun':6,'Jul':7,'Aug':8,'Sep':9, 'Oct':10, 'Nov': 11, 'Dec': 12,'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5}

In [34]:
detailed['Month'] = detailed['Month_name'].map(months)

In [35]:
detailed.sort_values(by=['movieID','Week_num']).to_csv('weekend-cleaned-2.csv',index=False)

In [37]:
movies = list(set(detailed['movieID']))

In [38]:
m_data = []

In [39]:
for m in movies:
    m_info = {}
    m_info['movieID'] = m

    #Week 1
    if len(detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 1)]) > 0:
        m_info['Year'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 1)].iloc[0]['Year']
        m_info['w_1_date'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 1)].iloc[0]['Date']
        m_info['w_1_rank'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 1)].iloc[0]['Rank']
        m_info['w_1_gross'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 1)].iloc[0]['WeekendGross']
    else:
        m_info['Year'] = np.nan
        m_info['w_1_date'] = np.nan
        m_info['w_1_rank'] = np.nan
        m_info['w_1_gross'] = np.nan
    

    #Week 2
    if len(detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 2)]) > 0:
        m_info['w_2_date'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 2)].iloc[0]['Date']
        m_info['w_2_rank'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 2)].iloc[0]['Rank']
        m_info['w_2_gross'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 2)].iloc[0]['WeekendGross']
        m_info['w2_$_change'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 2)].iloc[0]['pcChange']
    else:
        m_info['w_2_date'] = np.nan
        m_info['w_2_rank'] = np.nan
        m_info['w_2_gross'] = np.nan
        m_info['w2_$_change'] = np.nan
    m_info['w2_rank_change'] = m_info['w_1_rank'] - m_info['w_2_rank']

    #Week 3
    if len(detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 3)]) > 0:
        m_info['w_3_date'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 3)].iloc[0]['Date']
        m_info['w_3_rank'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 3)].iloc[0]['Rank']
        m_info['w_3_gross'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 3)].iloc[0]['WeekendGross']
        m_info['w3_$_change'] = detailed[(detailed['movieID'] == m) & (detailed['Week_num'] == 3)].iloc[0]['pcChange']
    else:
        m_info['w_3_date'] = np.nan
        m_info['w_3_rank'] = np.nan
        m_info['w_3_gross'] = np.nan
        m_info['w3_$_change'] = np.nan
    m_info['w3_rank_change'] = m_info['w_2_rank'] - m_info['w_3_rank']
    m_data.append(m_info)



In [41]:
import json
from pandas.io.json import json_normalize

In [42]:
njs = json_normalize(m_data)

In [45]:
njs.to_csv('weekend-tranformed.csv', index=False)

In [2]:
njs = pd.read_csv('weekend-tranformed.csv')

In [6]:
films = pd.read_csv('films_cleaned.csv')

In [7]:
f = films.merge(njs)